In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas
train_file_path = "/kaggle/input/quora-insincere-questions-classification/train.csv"
train_file = pandas.read_csv(train_file_path)
train_file.head()

In [ ]:
train_file.info()

In [ ]:
import re
import string
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())
remove_n = lambda x: re.sub("\n", " ", x)
remove_non_ascii = lambda x: re.sub(r'[^\x00-\x7f]',r' ', x)
train_file['question_text'] = train_file['question_text'].map(alphanumeric).map(punc_lower).map(remove_n).map(remove_non_ascii)
train_file['question_text'][0]

In [ ]:
from matplotlib import pyplot
import seaborn
import numpy
import pandas

data_rate = train_file['target'].sum()/train_file['target'].count()
data_rate

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
set(stopwords.words('english'))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

# train_data_1 = train_file[train_file['target']==1].iloc[:,:]
# train_data_0 = train_file[train_file['target']==0].iloc[:int(len(train_data_1)/1),:]
# train_data = pandas.concat([train_data_1,train_data_0],axis=0)

train_data = train_file

X_train = train_data.question_text
y_train = train_data.target

cv = CountVectorizer(ngram_range=((1,1)), stop_words='english')

X_train_cv = cv.fit_transform(X_train)

lr = LogisticRegression()

In [ ]:
lr.fit(X_train_cv, y_train)

In [ ]:
test_file_path = "/kaggle/input/quora-insincere-questions-classification/test.csv"
test_file = pandas.read_csv(test_file_path)
test_file.head()

In [ ]:
test_file['question_text'] = test_file['question_text'].map(alphanumeric).map(punc_lower).map(remove_n).map(remove_non_ascii)
test_file['question_text'][0]

In [ ]:
X_test_cv = cv.transform(test_file.question_text)

In [ ]:
result = lr.predict(X_test_cv)

In [ ]:
df = pandas.DataFrame()
df['qid'] = test_file['qid']
df['prediction'] = result
df.to_csv('submission.csv', index=False)